# Task: Research Assistant with Web Search Integration

## Overview

This task involves conducting research on a list of topics provided by the user. The task flow includes performing web searches to gather information on the latest news for each topic, generating relevant keywords, and utilizing various tools to accomplish the task efficiently.

### Task Flow

1. **Input**: 
   - The user provides a list of topics they want to research. Each topic is handled individually to gather relevant information.

2. **Web Search and Context**: 
   - For each topic, a web search is conducted using DuckDuckGo to find the latest news and information.
   
3. **Research Assistant**: 
   - The system analyzes the search results (HTML snippets) and generates a list of Wikipedia keywords related to the topic.
   - A prompt instructs the assistant to identify relevant keywords based on the search result content.

4. **Parallel Processing**: 
   - The task allows for parallel processing of topics to improve efficiency and speed up the keyword generation process.

This workflow allows for efficient gathering of research topics and relevant Wikipedia keywords, providing a comprehensive approach to in-depth topic exploration.


To recreate the notebook, a link to the Colab notebook is provided below. The notebook contains the code implementation for the task.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1C_yeSW4hAMLp0kNseXoxGpAA2cpC9DLu/view?usp=sharing)

For more details on the task or any questions, please feel free to reach out to the author, [Julep AI](hey@julep.ai).

Installing the Julep Client

In [1]:
!pip install julep -U --quiet

#### NOTE:

The uuid is generated for the agent and task, and is used to identify the agent and task in the system. Once the agent and task are created, the uuid should not be changed for simplicity.

Changing the uuid will cause the agent and task to be treated as a new agent and task respectively. The previous agent and task will still persist in the system.

In [2]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

In [3]:
from julep import Client

api_key = "" # Your API key here

# Create a client
client = Client(api_key=api_key, environment="dev")

## Creating an "agent"


Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [4]:
# Defining the agent
name = "Jarvis"
about = "The original AI conscious the Iron Man."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}


# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

## Listing all the agents

We list the agents to see what agents are available.

In [ ]:
# Listing all agents available in the account
get_agents = client.agents.list()
get_agents.items

We get the current agent ID from the list of agents.

In [ ]:
# get the agent id
agent_id = get_agents.items[0].id
agent_id

## Defining a Task

Tasks in Julep are Github Actions style workflows that define long-running, multi-step actions. 
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [7]:
import yaml

Here is a Task which uses a agent to generate a sarcastic response to a given text using a DuckDuckGo and Wikipedia tool.

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [8]:
task_def= yaml.safe_load("""
name: Research Assistant to find Wikipedia Keywords

input_schema:
  type: object
  properties:
    topics:
      type: array
      items:
        type: string
      description: The topics to search for.
    location:
      type: array
      items:
        type: string
      description: The loacation to search for.

tools:
- name: websearch
  type: integration
  integration:
    provider: duckduckgo_search

main:
- over: _.topics
  map:
    tool: websearch
    arguments:
      query: "'the latest news about ' + _"

- over: _
  parallelism: 2
  map:
    prompt:
    - role: system
      content: >-
        You are a research assistant.
        I need you to do in-depth research on topics trending in the news currently.
        Based on the following latest html news snippet, come up with a list of wikipedia keywords to search:
        "{{_}}"
        Your response should be a list of keywords, separated by commas. Do not add any other text.
        Example: `KEYWORDS: keyword1, keyword2, keyword3`

    unwrap: true
""")

Creating/Updating a task to generate a sarcastic response to a given text using a Intergation.

In [9]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

In [ ]:
#listing all tasks for a given agent
get_tasks = client.tasks.list(agent_id=AGENT_UUID)
get_tasks.items 

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates a execution worflow for the Task defined in the yaml file.

In [11]:
execution= client.executions.create(
    task_id=task.id,
    input={
        "topics": ["Burger King Cup on the Ground Behind a Wendy’s","Forbidden Chemical X","Finger Bracelets","Amusing Notions"]
    }
)

In [ ]:
execution.id

In [ ]:
# getting the execution details
execution = client.executions.get(execution.id)
#printing the output
execution.output

Streaming execution steps of the defined Task.

In [ ]:
client.executions.transitions.stream(execution_id=execution.id)

Listing the execution states.

In [ ]:
client.executions.transitions.list(execution_id=execution.id).items